Try to learn (1) as a piecewise linear constant curve

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from matplotlib_inline.backend_inline import set_matplotlib_formats
import os
from itertools import chain
set_matplotlib_formats('pdf', 'svg')

from deepthermal.FFNN_model import fit_FFNN, FFNN, init_xavier
from deepthermal.validation import create_subdictionary_iterator, k_fold_cv_grid, add_dictionary_iterators
from deepthermal.plotting import plot_result, plot_model_1d

from deep_reparametrization.plotting import plot_reparametrization
from deep_reparametrization.reparametrization import (
    get_elastic_metric_loss,
    compute_loss_reparam,
    get_elastic_error_func,
)
from deep_reparametrization.ResNET import ResNET
from deep_reparametrization.helpers import get_pc_curve, get_pl_curve
import experiments.curves as c1


In [ ]:
########
DIR = "../figures/curve_1_pl/"
SET_NAME = "curve_1_pl_exp_1"
PATH_FIGURES = os.path.join(DIR,SET_NAME)
if not os.path.exists(PATH_FIGURES):
    os.makedirs(PATH_FIGURES)
########


FOLDS = 1
N = 128 # training points internal

loss_func = get_elastic_metric_loss(r=get_pl_curve(c1.r,N), constrain_cost=1e3, verbose=False)
penalty_free_loss_func = get_elastic_metric_loss(r=c1.r, constrain_cost=0, verbose=False)

MODEL_PARAMS = {
    "input_dimension": [1],
    "output_dimension": [1],
    "activation": [ "tanh"],
    "n_hidden_layers": [2],#,8,16,64],
    "neurons": [2,8, 32, 128]
}
MODEL_PARAMS_2 = {
    "input_dimension": [1],
    "output_dimension": [1],
    "activation": [ "tanh"],
    "n_hidden_layers": [2,4,8,16,32,64],
    "neurons": [8],
}
# extend the previous dict with the zip of this
MODEL_PARAMS_EXPERIMENT = {
    "model": [FFNN, ResNET],
}
TRAINING_PARAMS = {
    "batch_size": [N],
    "regularization_param": [0],
    "compute_loss": [compute_loss_reparam],
    "loss_func": [loss_func],
}
# extend the previous dict with the zip of this
TRAINING_PARAMS_EXPERIMENT = {
    "optimizer": ["strong_wolfe"],
    "num_epochs": [10],
    "learning_rate": [ 1],
}


In [ ]:
# create iterators
model_params_iter_1= create_subdictionary_iterator(MODEL_PARAMS)
model_params_iter_2= create_subdictionary_iterator(MODEL_PARAMS_2)
model_params_iter = chain.from_iterable((model_params_iter_1, model_params_iter_2))

model_exp_iter = create_subdictionary_iterator(MODEL_PARAMS_EXPERIMENT, product=False)
exp_model_params_iter = add_dictionary_iterators(model_exp_iter, model_params_iter)

training_params_iter = create_subdictionary_iterator(TRAINING_PARAMS)
training_exp_iter = create_subdictionary_iterator(TRAINING_PARAMS_EXPERIMENT, product=False)
exp_training_params_iter = add_dictionary_iterators(training_exp_iter, training_params_iter)

In [ ]:
# Load data
x_train = torch.linspace(0, 1, N, requires_grad=True).unsqueeze(1)
q_pl = get_pc_curve(c1.q,N)
q_train = q_pl(x_train.detach())

data = TensorDataset(x_train, q_train)


Do the actual training

In [ ]:
cv_results = k_fold_cv_grid(
    model_params=exp_model_params_iter,
    fit=fit_FFNN,
    training_params=exp_training_params_iter,
    data=data,
    folds=FOLDS,
    verbose=True,
    trials=1
)

In [ ]:
# plotting

disc_points = x_train.detach()

plot_kwargs = {
    "x_test": disc_points,
    "x_train": disc_points,
    "y_train": c1.ksi(disc_points),
    "x_axis": "t",
    "y_axis": "$\\varphi(t)$",
    "compare_label" : "analytical solution (smooth)"
}
plot_result(
    path_figures=PATH_FIGURES,
    plot_name=SET_NAME,
    **cv_results,
    plot_function=plot_model_1d,
    function_kwargs=plot_kwargs,
)

In [ ]:


models = cv_results["models"]

parameters = np.vectorize(lambda model: sum(p.numel() for p in model.parameters()))(models).flatten()
model_type = np.vectorize(str)(models).flatten()
layers = np.vectorize(lambda model: model.n_hidden_layers)(models).flatten()
neurons = np.vectorize(lambda model: model.neurons)(models).flatten()
loss_array = np.vectorize(lambda model: penalty_free_loss_func(model, x_train, q_train).detach())(models).flatten()

# make data frame
d_results = pd.DataFrame({"loss": loss_array, "neurons": neurons, "layers":layers ,"parameters": parameters, "model": model_type})

d_results_layer = d_results[d_results.neurons == 8]
d_results_neurons = d_results[d_results.layers == 2]


In [ ]:
fig_neurons= sns.lineplot(data =d_results_neurons, y="loss",  x="neurons", hue="model", ci=80, err_style="bars")
fig_neurons.set(xscale="log",yscale="log", xlabel="Neurons")
fig_neurons.set(ylabel="$E$")
plt.savefig(f"{PATH_FIGURES}/neurons_error.pdf")
plt.show()



In [ ]:
print("faled, total:")
print(len(d_results_neurons[d_results_neurons.loss > 2]), len(d_results_neurons))

In [ ]:
fig_layers= sns.lineplot(data =d_results_layer, y="loss",  x="layers", hue="model", ci=80, err_style="bars")
fig_layers.set(yscale="log", xscale="log",xlabel="Layers", ylabel="Error")
fig_neurons.set(ylabel="$E$")
plt.savefig(f"{PATH_FIGURES}/layer_error.pdf")
plt.show()


In [ ]:
print("faled, total:")
print(len(d_results_layer[d_results_layer.loss
                          > 2]), len(d_results_layer))

In [ ]:
fig_scatter = sns.scatterplot(data=d_results, y="loss", x="parameters", hue="layers", style="model")
fig_neurons.set(ylabel="$E$")
fig_scatter.set(yscale="log")
plt.show()

In [ ]:
print(len(d_results[d_results.loss < 2]), len(d_results))